Table Verification Notebook
============

Performs 2 main tasks:
1. Flags table axis combinations which have column profiles (such as B probes) which have large enough 'Best to average poit removed fit quality' (see below). After these combinations have been flagged, their psi contours can be investigated by the user
2. Finds table axis combinations which do not exist in the table

$$\textrm{Best to average point removed fit quality} = \textrm{max}(\textrm{abs}(\chi_i - \bar{\chi}))$$
Where $\chi_i$ is the quadratic fit quality with point $i$ removed, and $\bar{\chi}$ is the average of all $\chi_i$ for a single profile

In [ ]:
from enum import Enum   

class System(Enum):
    PI3 = 1
    PI4 = 2
    FDP = 3

system_in_use = System.PI3

Get profile names to best to average point removed fit quality of

In [ ]:
check_b_probe_colummns = True
other_columns = ['WBPol', 'q020', 'q050', 'q080', 'WBPolNoDCInFC', 'phiPlInFC']

In [ ]:
import os
import sys

sys.path.append(os.getenv("AURORA_REPOS"))

from flagships.post_processing import GenerateColumns

if(system_in_use == System.PI3):
    csv_generator =  GenerateColumns.PI3CSVGenerator()
elif(system_in_use == System.PI4):
    csv_generator =  GenerateColumns.PI4CSVGenerator()
elif(system_in_use == System.FDP):
    csv_generator = GenerateColumns.FDPCase22a_CSVGenerator()
else:
    print('Ensure correct system is set')

b_col_names = []
if check_b_probe_colummns:
    for val in csv_generator.add_bprobe_columns([]):
        b_col_names.append('B' + val[0])

profile_columns_of_interest = other_columns + b_col_names
existent_columns_of_interest = []

all_cols = csv_generator.create_all_columns()
all_col_names = [col.Name for col in all_cols]

for col_of_interest in profile_columns_of_interest:
    if(all_col_names.count(col_of_interest) != 0):
        existent_columns_of_interest.append(col_of_interest)

print('Found the following columns: ', existent_columns_of_interest)

Create verification object

In [ ]:
table_name = "pi3b_asbuilt_pfc17500ab_2022-06-09"

In [ ]:

backend_cs_dll = os.path.join('bin', 'Debug')
sys.path.append(backend_cs_dll)

existent_columns_of_interest = ['B161087','B211100','B261087','B291060','B215008','B261008','B291008']

import clr
clr.AddReference('table_verification_cs')

from TableVerification import TableVerifier, FailingProfile

# tv = TableVerifier(table_name, ','.join(existent_columns_of_interest))
# tv = TableVerifier("pi3b_asbuilt_pfc17500ab_2022-06-09", "B161087,B211100,B261087,B291060,B215008,B261008,B291008");
verifier = TableVerifier("pi3b_asbuilt_pfc17500ab_2022-06-09", "B161087,B211100,B261087,B291060,B215008,B261008,B291008")
# print(f"Read {tv.GetNumRowsInDatatable()} Rows")


#### Flag abnormal profiles and view corresponding 5% psi contours

First go through the table and score each profile for each table axis combination based on the best to average point removed fit quality

In [ ]:
verifier.GenerateProfileScores()

Use different thresholds to change how many profiles and table axis combinations would be flagged

In [ ]:
def find_flags_from_threshold(tv : TableVerifier, columns_of_interest : list[str], threshold : float):
    profile_scores = tv.GetProfileScoreTable()

    select_str = ''
    for col in columns_of_interest:
        select_str += f'{col} > {threshold} OR '
    select_str = select_str[:-3]

    failing_row_arr = profile_scores.Select(select_str)
    failing_rows = profile_scores.Clone()

    num_fails = 0
    for row in failing_row_arr:
        for col in columns_of_interest:
            if row[col] > threshold:
                num_fails += 1
        failing_rows.ImportRow(row)

    print(f'Table has {failing_rows.Rows.Count} flagged table axis configurations')
    print(f'Table has {num_fails} flagged profiles')

    axes_names = [col.ColumnName for col in failing_rows.Columns][0:7]

    cs_float_type = type(row[columns_of_interest[0]])

    print('\nNumber of instances where certain table axes caused flag:')

    # the search axis col is of type db.null (not float)
    for axis_name in axes_names:
        num_fails = 0
        for row in failing_rows.Rows:
            if type(row[axis_name]) != cs_float_type:
                num_fails += 1
        print(f'{axis_name}: {num_fails}')

In [ ]:
find_flags_from_threshold(verifier, existent_columns_of_interest, 0.4)
# Note that in a single row, multiple profiles can be flagged

In [ ]:
threshold = 0.4
failing_profiles = verifier.GetFailingProfiles(threshold)
print('Found ', len(failing_profiles) , ' failing profiles')

Plot psi contours for flagging table axis configurations

In [ ]:
from fileinput import filename
import sys, os
import matplotlib.pyplot as plt

sys.path.append(os.getenv("AURORA_REPOS"))

from flagships.post_processing.GenerateColumns import get_full_file_path
from flagships.post_processing.ParseFlagshipsFile import FlagshipsParser
from flagships.post_processing.EquilibriumPlotting import PlotPsi5PctContours
        
def plot_unique_fail_contours(failing_profiles : list[FailingProfile], axes):

    unique_table_axes_combos = []
    unique_table_axes_offending_names_and_values = []
    for failing_profile in failing_profiles:
        tbl_vals = {}
        for pr in failing_profile.TableParams:
            tbl_vals[pr.Key] = pr.Value
        already_in_dict = False
        for unique_table_axes_combo in unique_table_axes_combos:
            if unique_table_axes_combo == tbl_vals:
                already_in_dict = True
                break
        if not already_in_dict:
            unique_table_axes_combos.append(tbl_vals)
            unique_table_axes_offending_names_and_values.append((failing_profile.SearchAxis, failing_profile.TableParams[failing_profile.SearchAxis]))

    filesLocation = failing_profiles[0].FilesLocation

    for table_params, offending_name_and_value in zip(unique_table_axes_combos, unique_table_axes_offending_names_and_values):
        search_axis_name = offending_name_and_value[0]
        offending_value = offending_name_and_value[1]
        search_axis_values = list(axes[search_axis_name])

        if search_axis_values.index(offending_value) == 0:
            offending_values = search_axis_values[:2]
            offending_index = 0
        elif search_axis_values.index(offending_value) == len(search_axis_values)-1:
            offending_values = search_axis_values[len(search_axis_values)-2:]
            offending_index = 1
        else:
            offending_values = search_axis_values[search_axis_values.index(offending_value)-1:search_axis_values.index(offending_value)+2]
            offending_index = 1

        i = 1
        plt.figure(figsize=(12, 6))
        for val in offending_values:
            table_params[search_axis_name] = val
            suffix = "." + os.path.normpath(filesLocation).split(os.sep)[-2]
            prefix = ""
            lut_root = os.path.normpath(os.getenv('FS_ARCHIVE_ROOT') + os.path.sep + filesLocation)
            filepath = get_full_file_path(lut_root=lut_root, prefix=prefix, suffix=suffix,
                    soak = table_params['psieq_soak'], beta=table_params['beta_pol1_setpoint'], dc=table_params['psieq_dc'], NevinsC=table_params['NevinsC'], 
                    NevinsAB=table_params['NevinsA'], NevinsN=table_params['NevinsN'], current_ratio=table_params['Ipl_setpoint']*1e-6)
            filepath = filepath.replace('\\', os.path.sep)
            if(not os.path.exists(filepath)):
                filepath = filepath[:filepath.find('ab', filepath.find('equil'))+3] + filepath[filepath.find('ab', filepath.find('equil'))+4:]
            if(not os.path.exists(filepath)):
                filepath = filepath[:filepath.rfind('ab')+3] + filepath[filepath.rfind('ab')+4:]
            if(not os.path.exists(filepath)):
                print(f'File cannot be found:\n {filepath}')
                continue

            print(filepath)
            parser = FlagshipsParser(filepath, filepath)
            plt.subplot(1, len(offending_values), i)
            PlotPsi5PctContours(parser)
            plt.scatter(parser.GetScalar('xpoint_r'), parser.GetScalar('xpoint_z'), marker='x') #TODO add this as a function to EquilmPlotting.py, but points dont look right
            title = search_axis_name + f" = {val}"
            if i-1 == offending_index :
                title += "\n Offending Value"
            plt.title(title)
            i += 1
            
        plt.show()
        in_key = input("Press Enter to continue...")
        if(in_key == 'end'):
            break


In [ ]:
plot_unique_fail_contours(failing_profiles, verifier.TableAxesValues)

#### Check for non existent table axes combinations
<sub><sup>Currently takes about 15 minutes</sup></sub>

In [72]:
hole_configurations = verifier.CheckForHoles()